In [1]:
import re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import string
import json
import shutil
import os
import pandas as pd
from itertools import combinations

In [2]:
countryList = ['USA', 'United States', 'Canada', 'Britain', 'England', 'Germany', 'France', 'Italy',
              'China', 'Korea', 'Japan', 'Sweden', 'Brazilia', 'Australia', 'UK', 'Portugal', 'United Kingdom',
              'Finland', 'Norway', 'Ireland', 'Netherlands', 'Ecuador']

In [3]:
instituteList = ['College', 'Institute', 'University', 'Group', 'School', 'Department', 'Microsoft', 'Google',
                'Adobe', 'Disney', 'Facebook', 'Tableau', 'UC', 'Inc.', 'Laboratory', 'Center', 'Autodesk', 'Lab',
                'Inria', 'Univ']

In [4]:
Asia = ['China', 'Singapore', 'Korea', 'India', 'Lebanon', 'Japan', 'Israel', 
        'Hong Kong', 'Taiwan', 'Malaysia', 'Turkey', 'Qatar', 'UAE', 'Philippines',
       'Pakistan', 'Vietnam', 'Saudi Arabia', 'Iran']

NorthAmerica = ['USA', 'Canada','Costa Rica']

LatinAmerica = ['Brazil', 'Mexico', 'Ecuador','Paraguay','Argentina']

Europe = ['Germany', 'Sweden', 'Belgium', 'UK', 'Spain', 'Italy', 
          'France', 'Finland', 'Slovenia', 'Denmark', 'Switzerland', 
          'Austria','Greece', 'Bulgaria', 'Netherlands','Portugal', 
          'Ireland', 'Romania', 'Norway', 'Cyprus', 'Hungary']

Oceanic = ['Australia', 'New Zealand']

Africa = ['Namibia', 'Egypt', 'South Africa', 'Rwanda', 'Tanzania','Uganda']

In [5]:
ctyColor = {'Asia': '#fffac8', 'NorthAmerica': '#e6194B', 
            'LatinAmerica': '#fabebe', 'Europe': '#42d4f4', 
            'Oceanic': '#a9a9a9', 'Africa': '#aaffc3'}

In [6]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    # only read the 1st page
    pagenos=[0]

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [7]:
def text2data(file):
    try:
        res = convert_pdf_to_txt(file)
        print ('Finished converting PDF')
    except TypeError:
        print (file)
        res = ''
    new_res = res.split('\n')
    info = []
    new_info = []
    for idx, ele in enumerate (new_res):
        if idx + 1 < len(new_res):
            # the title, authors and affiliations are likely to be before ABSTRACT
            if "ABSTRACT" in new_res[idx + 1]:
                info = new_res[0:idx]
                for ele in info:
                    # Remove numbers
                    ele = re.sub(r'\d+', '', ele)
                    # Remove empty ones
                    if len(ele) > 0 and not ele.isspace():
                        # skip the emails
#                         if '@' in ele or '{' in ele:
#                             continue
#                         else:
                        new_info.append(ele)
    # Extract fields 
    data = {'aff': [], 'country': []}
    for ele in new_info:
        # Extract countries
        for country in countryList:
            if country in ele:
                data['country'].append(country)
        
        # Extract institutes
        for inst in instituteList:
            if inst in ele:
                for country in countryList:
                    if country in ele:
                        ele = ele.replace(country, '')
                ele = ele.strip(' ')
#                 ele = ele.translate(string.punctuation)
                        
                data['aff'].append(ele)
        
        # Extract authors
        
#         # Titles are unlikely to include commas and longer 
#         if ',' not in ele and len(ele) >= 4:
#             data['title'].append(ele)
#         # Search for names
#         else:
#             ele = ele.split(',')
#             for name in ele:
#                 name = name.strip(' ')
#                 print (name)
            
#         m.search_first_name('Mikael')
#         m.search_last_name('Remy')
#     print (new_info)
    return data


import os
rootDir = '/Users/luhe/Desktop/research/network_analysis/data/chi19-main/N/'
count = 0
new_data = {}
subFolder = rootDir.split('/')[-2]
year = rootDir.split('/')[-3]
outFile = year + subFolder + '.txt'
for dirName, subdirList, fileList in os.walk(rootDir):
    print (len(fileList))
    for fname in fileList:
        if fname not in new_data and fname.endswith('.pdf'):
            count += 1
            print (fname + '    ' + str(count))
            new_data[fname] = text2data(rootDir + fname)
            
#         except TypeError:
        else:
            continue
#         fout.write(''.join(data[fname]) + '\n')
outFile = '/Users/luhe/Desktop/research/network_analysis/output/2019/' + outFile
with open(outFile, 'w') as fout:
    print ('Writing: ' + outFile)
    fout.write('file' + '\t' + 'affiliations' + '\t' + 'country' + '\n')
    for key in new_data:
        fout.write(key + '\t')
        affiliations = new_data[key]['aff']
        country = new_data[key]['country']
        fout.write('-'.join(affiliations))
        fout.write('\t')
        fout.write('-'.join(country))
        fout.write('\n')

In [41]:
rootDir = '/Users/luhe/Desktop/research/network_analysis/data/chi17-main/'
count = 0
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        if count < 50:
            src = rootDir + fname
            dst = rootDir + 'A/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)
#             print (src + ' moved to: ' + dst)
        elif count >=50 and count < 100:
            src = rootDir + fname
            dst = rootDir + 'B/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)
#             print (src + ' moved to: ' + dst)
        elif count >= 100 and count < 150:
            src = rootDir + fname
            dst = rootDir + 'C/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 150 and count < 200:
            src = rootDir + fname
            dst = rootDir + 'D/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 200 and count < 250:
            src = rootDir + fname
            dst = rootDir + 'E/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 250 and count < 300:
            src = rootDir + fname
            dst = rootDir + 'F/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 300 and count < 350:
            src = rootDir + fname
            dst = rootDir + 'G/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 350 and count < 400:
            src = rootDir + fname
            dst = rootDir + 'H/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 400 and count < 450:
            src = rootDir + fname
            dst = rootDir + 'I/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 450 and count < 500:
            src = rootDir + fname
            dst = rootDir + 'J/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 500 and count < 550:
            src = rootDir + fname
            dst = rootDir + 'K/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >= 550 and count < 600:
            src = rootDir + fname
            dst = rootDir + 'L/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)
        elif count >=600:
            src = rootDir + fname
            dst = rootDir + 'M/'
            shutil.copy(src, dst)
            count += 1
            os.remove(src)	
#             print (src + ' moved to: ' + dst)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/luhe/Desktop/research/network_analysis/data/chi16-main/p1374-echtler.pdf'

In [42]:
rootDir = '/Users/luhe/Desktop/research/network_analysis/data/chi1-main/'
count = 0
for dirName, subdirList, fileList in os.walk(rootDir):
    count += len(fileList)
print (count)

547


## Prepare data for network analysis

In [20]:
inFile16A = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainA.txt'
inFile16B = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainB.txt'
inFile16C = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainC.txt'
inFile16D = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainD.txt'
inFile16E = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainE.txt'
inFile16F = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainF.txt'
inFile16G = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainG.txt'
inFile16H = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainH.txt'
inFile16I = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainI.txt'
inFile16J = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainJ.txt'
inFile16K = '/Users/luhe/Desktop/research/network_analysis/output/2016/chi16-mainK.txt'

In [21]:
df16A = pd.read_csv(inFile16A, sep='\t')
df16B = pd.read_csv(inFile16B, sep='\t')
df16C = pd.read_csv(inFile16C, sep='\t')
df16D = pd.read_csv(inFile16D, sep='\t')
df16E = pd.read_csv(inFile16E, sep='\t')
df16F = pd.read_csv(inFile16F, sep='\t')
df16G = pd.read_csv(inFile16G, sep='\t')
df16H = pd.read_csv(inFile16H, sep='\t')
df16I = pd.read_csv(inFile16I, sep='\t')
df16J = pd.read_csv(inFile16J, sep='\t')
df16K = pd.read_csv(inFile16K, sep='\t')



df16 = df16A.append(df16B)
df16 = df16.append(df16C)
df16 = df16.append(df16D)
df16 = df16.append(df16E)
df16 = df16.append(df16F)
df16 = df16.append(df16G)
df16 = df16.append(df16H)
df16 = df16.append(df16I)
df16 = df16.append(df16J)
df16 = df16.append(df16K)

In [67]:
country16 = df16['country']
aff = df16['affiliations']

In [68]:
newCountry16 = []
for ele in country16:
    ele = ele.replace('Scotland','UK')
    newCountry16.append(ele)

In [91]:
tplCount = {}
isolates = {}
for ele in newCountry16:
    # print (ele)
    if '-' in ele:
        tpl = ele.split('-')
        tpl = list(set(tpl))
        # exists international collboration
        if len(tpl) > 1:
        # Get all permutations of [1, 2, 3] 
            comb = combinations(tpl, 2) 
            for i in list(comb): 
                if i not in tplCount:
                    tplCount[i] = 1
                else:
                    tplCount[i] += 1 
with open('./output/2016All.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in tplCount:
        val = tplCount[key]
        fout.write('\t'.join([key[0],key[1],str(val)]))
        fout.write('\n')
    

In [92]:
tplCount = {}
idList= []
count = 0
for ele in newCountry16:
        if '-' in ele:
            tpl = ele.split('-')
            tpl = list(set(tpl))
            # exists international collboration
            if len(tpl) > 1:
            # Get all permutations of [1, 2, 3] 
                comb = combinations(tpl, 2) 
                for i in list(comb):
                    if i not in tplCount:
                        tplCount[i] = 1
                    else:
                        tplCount[i] += 1

                        
keyList = list(tplCount)
for key in tplCount:
    if reversed(key) in keyList:
        print (key)
print (keyList)

[('Canada', 'USA'), ('Denmark', 'Finland'), ('Romania', 'USA'), ('USA', 'UK'), ('Denmark', 'Ireland'), ('Denmark', 'UK'), ('Ireland', 'UK'), ('Germany', 'UK'), ('Austria', 'Canada'), ('Sweden', 'USA'), ('Mexico', 'USA'), ('USA', 'Hong Kong'), ('Germany', 'Sweden'), ('Germany', 'Belgium'), ('Germany', 'USA'), ('Belgium', 'USA'), ('Singapore', 'Hong Kong'), ('Germany', 'Canada'), ('Spain', 'UK'), ('Korea', 'Canada'), ('Brazil', 'Canada'), ('Sweden', 'Italy'), ('France', 'Canada'), ('Italy', 'Finland'), ('Slovenia', 'Canada'), ('Slovenia', 'UK'), ('Canada', 'UK'), ('Germany', 'Denmark'), ('India', 'USA'), ('Finland', 'UK'), ('Switzerland', 'USA'), ('Austria', 'USA'), ('Spain', 'Switzerland'), ('China', 'UK'), ('France', 'UK'), ('France', 'USA'), ('UK', 'USA'), ('Lebanon', 'UK'), ('Canada', 'Belgium'), ('Japan', 'USA'), ('USA', 'France'), ('France', 'Hong Kong'), ('Brazil', 'USA'), ('New Zealand', 'Canada'), ('Korea', 'USA'), ('New Zealand', 'Australia'), ('China', 'USA'), ('India', 'UK'),

### Creating country ID dictionary

In [93]:
ctyID = {}
count = 0
for pair in keyList:
    for ele in pair:
        if ele not in ctyID:
            ctyID[ele] = {'ID': count, 'color': ''}
            ctyID[ele]['ID'] = count
            if ele in Asia:
                ctyID[ele]['color'] = ctyColor['Asia']
            elif ele in NorthAmerica:
                ctyID[ele]['color'] = ctyColor['NorthAmerica']
            elif ele in LatinAmerica:
                ctyID[ele]['color'] = ctyColor['LatinAmerica']             
            elif ele in Europe:
                ctyID[ele]['color'] = ctyColor['Europe']
            elif ele in Oceanic:
                ctyID[ele]['color'] = ctyColor['Oceanic']
            elif ele in Africa:
                ctyID[ele]['color'] = ctyColor['Africa']
            count += 1


### Making a new dictionary with numbered ID keys

In [94]:
newtplCount = {}
for key in tplCount:
    newKey = (ctyID[key[0]]['ID'], ctyID[key[1]]['ID'])
    newtplCount[newKey] = tplCount[key]
#     del tplCount[key]

### Writing to Files

In [95]:
with open('./output/2016AllEdgesNumbered.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in newtplCount:
        val = newtplCount[key]
        fout.write('\t'.join([str(key[0]),str(key[1]),str(val)]))
        fout.write('\n')

### Compute publication volumn for each country

In [96]:
pubCount = {}
for ele in newCountry16:
        if '-' not in ele:
            if ele not in pubCount:
                pubCount[ele] = 1
            else:
                pubCount[ele] += 1
        else:
            lst = ele.split('-')
            for cty in lst:
                if cty not in pubCount:
                    pubCount[cty] = 1
                else:
                    pubCount[cty] += 1
for ele in pubCount:
    if ele in ctyID:
        ctyID[ele]['pubCount'] = pubCount[ele]

with open('./output/2016AllcountryID.txt', 'w') as fout:
    fout.write('Country' + '\t' + 'ID' + '\t' + 'Color' + '\t' + 'pubCount' + '\n')
    for key in ctyID:
        fout.write(key + '\t' + str(ctyID[key]['ID']) + '\t' + str(ctyID[key]['color']) + '\t' + str(ctyID[key]['pubCount']) + '\n')
print (ctyID)

{'Canada': {'ID': 0, 'color': '#e6194B', 'pubCount': 69}, 'USA': {'ID': 1, 'color': '#e6194B', 'pubCount': 385}, 'Denmark': {'ID': 2, 'color': '#42d4f4', 'pubCount': 21}, 'Finland': {'ID': 3, 'color': '#42d4f4', 'pubCount': 24}, 'Romania': {'ID': 4, 'color': '#42d4f4', 'pubCount': 2}, 'UK': {'ID': 5, 'color': '#42d4f4', 'pubCount': 157}, 'Ireland': {'ID': 6, 'color': '#42d4f4', 'pubCount': 4}, 'Germany': {'ID': 7, 'color': '#42d4f4', 'pubCount': 52}, 'Austria': {'ID': 8, 'color': '#42d4f4', 'pubCount': 5}, 'Sweden': {'ID': 9, 'color': '#42d4f4', 'pubCount': 22}, 'Mexico': {'ID': 10, 'color': '#fabebe', 'pubCount': 1}, 'Hong Kong': {'ID': 11, 'color': '#fffac8', 'pubCount': 6}, 'Belgium': {'ID': 12, 'color': '#42d4f4', 'pubCount': 6}, 'Singapore': {'ID': 13, 'color': '#fffac8', 'pubCount': 3}, 'Spain': {'ID': 14, 'color': '#42d4f4', 'pubCount': 5}, 'Korea': {'ID': 15, 'color': '#fffac8', 'pubCount': 31}, 'Brazil': {'ID': 16, 'color': '#fabebe', 'pubCount': 3}, 'Italy': {'ID': 17, 'color

### Compute descriptives

ctyID: {country: {ID: id, color: color}}
tplCount: {(c1,c2): count}

In [75]:
count = 0
for ele in country:
    if '-' in ele:
        tpl = set(ele.split('-'))
        if len(tpl) > 1:
            count += 1
        
print (count)

135


## 2017 analysis

In [23]:
inFile17A = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainA.txt'
inFile17B = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainB.txt'
inFile17C = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainC.txt'
inFile17D = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainD.txt'
inFile17E = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainE.txt'
inFile17F = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainF.txt'
inFile17G = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainG.txt'
inFile17H = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainH.txt'
inFile17I = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainI.txt'
inFile17J = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainJ.txt'
inFile17K = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainK.txt'
inFile17L = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainL.txt'
inFile17M = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainM.txt'
inFile17N = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainN.txt'
inFile17O = '/Users/luhe/Desktop/research/network_analysis/output/2017/chi17_mainO.txt'

In [24]:
df17A = pd.read_csv(inFile17A, sep='\t')
df17B = pd.read_csv(inFile17B, sep='\t')
df17C = pd.read_csv(inFile17C, sep='\t')
df17D = pd.read_csv(inFile17D, sep='\t')
df17E = pd.read_csv(inFile17E, sep='\t')
df17F = pd.read_csv(inFile17F, sep='\t')
df17G = pd.read_csv(inFile17G, sep='\t')
df17H = pd.read_csv(inFile17H, sep='\t')
df17I = pd.read_csv(inFile17I, sep='\t')
df17J = pd.read_csv(inFile17J, sep='\t')
df17K = pd.read_csv(inFile17K, sep='\t')
df17L = pd.read_csv(inFile17L, sep='\t')
df17M = pd.read_csv(inFile17M, sep='\t')
df17N = pd.read_csv(inFile17N, sep='\t')
df17O = pd.read_csv(inFile17O, sep='\t')



df17 = df17A.append(df17B)
df17 = df17.append(df17C)
df17 = df17.append(df17D)
df17 = df17.append(df17E)
df17 = df17.append(df17F)
df17 = df17.append(df17G)
df17 = df17.append(df17H)
df17 = df17.append(df17I)
df17 = df17.append(df17J)
df17 = df17.append(df17K)
df17 = df17.append(df17L)
df17 = df17.append(df17M)
df17 = df17.append(df17N)
df17 = df17.append(df17O)

In [76]:
country17 = df17['country']
country17.size
country.head()

0        USA-USA
1             UK
2             UK
3     USA-Canada
4    USA-USA-USA
Name: country, dtype: object

In [77]:
newCountry17 = []
for ele in country17:
    ele = ele.replace('Scotland','UK')
    newCountry17.append(ele)

In [78]:
tplCount = {}
isolates = {}
for idx, ele in enumerate(newCountry17):
    # print (idx, ele)
    if '-' in ele:
        tpl = ele.split('-')
        tpl = list(set(tpl))
        # exists international collboration
        if len(tpl) > 1:
        # Get all permutations of [1, 2, 3] 
            comb = combinations(tpl, 2) 
            for i in list(comb): 
                if i not in tplCount:
                    tplCount[i] = 1
                else:
                    tplCount[i] += 1 
with open('./output/2017All.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in tplCount:
        val = tplCount[key]
        fout.write('\t'.join([key[0],key[1],str(val)]))
        fout.write('\n')

In [79]:
keyList17 = list(tplCount)
ctyID = {}
count = 0
for pair in keyList17:
    for ele in pair:
        if ele not in ctyID:
            ctyID[ele] = {'ID': count, 'color': ''}
            ctyID[ele]['ID'] = count
            if ele in Asia:
                ctyID[ele]['color'] = ctyColor['Asia']
            elif ele in NorthAmerica:
                ctyID[ele]['color'] = ctyColor['NorthAmerica']
            elif ele in LatinAmerica:
                ctyID[ele]['color'] = ctyColor['LatinAmerica']             
            elif ele in Europe:
                ctyID[ele]['color'] = ctyColor['Europe']
            elif ele in Oceanic:
                ctyID[ele]['color'] = ctyColor['Oceanic']
            elif ele in Africa:
                ctyID[ele]['color'] = ctyColor['Africa']
            count += 1

In [261]:
newtplCount = {}
for key in tplCount:
    newKey = (ctyID[key[0]]['ID'], ctyID[key[1]]['ID'])
    newtplCount[newKey] = tplCount[key]

In [80]:
with open('./output/2017AllEdgesNumbered.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in newtplCount:
        val = newtplCount[key]
        fout.write('\t'.join([str(key[0]),str(key[1]),str(val)]))
        fout.write('\n')

In [97]:
pubCount = {}
for ele in newCountry17:
        if '-' not in ele:
            if ele not in pubCount:
                pubCount[ele] = 1
            else:
                pubCount[ele] += 1
        else:
            lst = ele.split('-')
            for cty in lst:
                if cty not in pubCount:
                    pubCount[cty] = 1
                else:
                    pubCount[cty] += 1
for ele in pubCount:
    if ele in ctyID:
        ctyID[ele]['pubCount'] = pubCount[ele]

with open('./output/2017AllcountryID.txt', 'w') as fout:
    fout.write('Country' + '\t' + 'ID' + '\t' + 'Color' + '\t' + 'pubCount' + '\n')
    for key in ctyID:
        fout.write(key + '\t' + str(ctyID[key]['ID']) + '\t' + str(ctyID[key]['color']) + '\t' + str(ctyID[key]['pubCount']) + '\n')
print (ctyID)

{'Canada': {'ID': 0, 'color': '#e6194B', 'pubCount': 67}, 'USA': {'ID': 1, 'color': '#e6194B', 'pubCount': 398}, 'Denmark': {'ID': 2, 'color': '#42d4f4', 'pubCount': 19}, 'Finland': {'ID': 3, 'color': '#42d4f4', 'pubCount': 19}, 'Romania': {'ID': 4, 'color': '#42d4f4', 'pubCount': 2}, 'UK': {'ID': 5, 'color': '#42d4f4', 'pubCount': 179}, 'Ireland': {'ID': 6, 'color': '#42d4f4', 'pubCount': 2}, 'Germany': {'ID': 7, 'color': '#42d4f4', 'pubCount': 68}, 'Austria': {'ID': 8, 'color': '#42d4f4', 'pubCount': 9}, 'Sweden': {'ID': 9, 'color': '#42d4f4', 'pubCount': 19}, 'Mexico': {'ID': 10, 'color': '#fabebe', 'pubCount': 1}, 'Hong Kong': {'ID': 11, 'color': '#fffac8', 'pubCount': 6}, 'Belgium': {'ID': 12, 'color': '#42d4f4', 'pubCount': 2}, 'Singapore': {'ID': 13, 'color': '#fffac8', 'pubCount': 4}, 'Spain': {'ID': 14, 'color': '#42d4f4', 'pubCount': 6}, 'Korea': {'ID': 15, 'color': '#fffac8', 'pubCount': 23}, 'Brazil': {'ID': 16, 'color': '#fabebe', 'pubCount': 4}, 'Italy': {'ID': 17, 'color

In [264]:
count = 0
for ele in country17:
    if '-' in ele:
        tpl = set(ele.split('-'))
        if len(tpl) > 1:
            count += 1
        
print (count)

157


In [26]:
df1617 = df16.append(df17)

In [27]:
country1617 = df1617['country']
country1617.size

1145

In [268]:
tplCount = {}
isolates = {}
for idx, ele in enumerate(country1617):
    # print (idx, ele)
    if '-' in ele:
        tpl = ele.split('-')
        tpl = list(set(tpl))
        # exists international collboration
        if len(tpl) > 1:
        # Get all permutations of [1, 2, 3] 
            comb = combinations(tpl, 2) 
            for i in list(comb): 
                if i not in tplCount:
                    tplCount[i] = 1
                else:
                    tplCount[i] += 1 
with open('./output/1617All.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in tplCount:
        val = tplCount[key]
        fout.write('\t'.join([key[0],key[1],str(val)]))
        fout.write('\n')

In [269]:
keyList1617 = list(tplCount)
ctyID = {}
count = 0
for pair in keyList1617:
    for ele in pair:
        if ele not in ctyID:
            ctyID[ele] = {'ID': count, 'color': ''}
            ctyID[ele]['ID'] = count
            if ele in Asia:
                ctyID[ele]['color'] = ctyColor['Asia']
            elif ele in NorthAmerica:
                ctyID[ele]['color'] = ctyColor['NorthAmerica']
            elif ele in LatinAmerica:
                ctyID[ele]['color'] = ctyColor['LatinAmerica']             
            elif ele in Europe:
                ctyID[ele]['color'] = ctyColor['Europe']
            elif ele in Oceanic:
                ctyID[ele]['color'] = ctyColor['Oceanic']
            elif ele in Africa:
                ctyID[ele]['color'] = ctyColor['Africa']
            count += 1

In [270]:
newtplCount = {}
for key in tplCount:
    newKey = (ctyID[key[0]]['ID'], ctyID[key[1]]['ID'])
    newtplCount[newKey] = tplCount[key]

In [271]:
with open('./output/1617AllEdgesNumbered.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in newtplCount:
        val = newtplCount[key]
        fout.write('\t'.join([str(key[0]),str(key[1]),str(val)]))
        fout.write('\n')

In [272]:
pubCount = {}
for ele in country1617:
        if '-' not in ele:
            if ele not in pubCount:
                pubCount[ele] = 1
            else:
                pubCount[ele] += 1
        else:
            lst = ele.split('-')
            for cty in lst:
                if cty not in pubCount:
                    pubCount[cty] = 1
                else:
                    pubCount[cty] += 1
for ele in pubCount:
    if ele in ctyID:
        ctyID[ele]['pubCount'] = pubCount[ele]

with open('./output/1617AllcountryID.txt', 'w') as fout:
    fout.write('Country' + '\t' + 'ID' + '\t' + 'Color' + '\t' + 'pubCount' + '\n')
    for key in ctyID:
        fout.write(key + '\t' + str(ctyID[key]['ID']) + '\t' + str(ctyID[key]['color']) + '\t' + str(ctyID[key]['pubCount']) + '\n')
print (ctyID)

{'USA': {'ID': 0, 'color': '#e6194B', 'pubCount': 783}, 'Canada': {'ID': 1, 'color': '#e6194B', 'pubCount': 136}, 'Finland': {'ID': 2, 'color': '#42d4f4', 'pubCount': 43}, 'Denmark': {'ID': 3, 'color': '#42d4f4', 'pubCount': 40}, 'Romania': {'ID': 4, 'color': '#42d4f4', 'pubCount': 4}, 'UK': {'ID': 5, 'color': '#42d4f4', 'pubCount': 331}, 'Ireland': {'ID': 6, 'color': '#42d4f4', 'pubCount': 6}, 'Germany': {'ID': 7, 'color': '#42d4f4', 'pubCount': 120}, 'Austria': {'ID': 8, 'color': '#42d4f4', 'pubCount': 14}, 'Sweden': {'ID': 9, 'color': '#42d4f4', 'pubCount': 41}, 'Mexico': {'ID': 10, 'color': '#fabebe', 'pubCount': 1}, 'Hong Kong': {'ID': 11, 'color': '#fffac8', 'pubCount': 12}, 'Belgium': {'ID': 12, 'color': '#42d4f4', 'pubCount': 8}, 'Singapore': {'ID': 13, 'color': '#fffac8', 'pubCount': 7}, 'Spain': {'ID': 14, 'color': '#42d4f4', 'pubCount': 11}, 'Korea': {'ID': 15, 'color': '#fffac8', 'pubCount': 54}, 'Brazil': {'ID': 16, 'color': '#fabebe', 'pubCount': 7}, 'Italy': {'ID': 17, '

## 2018 Analysis

In [8]:
inFile18A = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainA.txt'
inFile18B = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainB.txt'
inFile18C = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainC.txt'
inFile18D = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainD.txt'
inFile18E = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainE.txt'
inFile18F = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainF.txt'
inFile18G = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainG.txt'
inFile18H = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainH.txt'
inFile18I = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainI.txt'
inFile18J = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainJ.txt'
inFile18K = '/Users/luhe/Desktop/research/network_analysis/output/2018/chi18-mainK.txt'


In [9]:
df18A = pd.read_csv(inFile18A, sep='\t')
df18B = pd.read_csv(inFile18B, sep='\t')
df18C = pd.read_csv(inFile18C, sep='\t')
df18D = pd.read_csv(inFile18D, sep='\t')
df18E = pd.read_csv(inFile18E, sep='\t')
df18F = pd.read_csv(inFile18F, sep='\t')
df18G = pd.read_csv(inFile18G, sep='\t')
df18H = pd.read_csv(inFile18H, sep='\t')
df18I = pd.read_csv(inFile18I, sep='\t')
df18J = pd.read_csv(inFile18J, sep='\t')
df18K = pd.read_csv(inFile18K, sep='\t')

In [10]:
df18 = df18A.append(df18B)
df18 = df18.append(df18C)
df18 = df18.append(df18D)
df18 = df18.append(df18E)
df18 = df18.append(df18F)
df18 = df18.append(df18G)
df18 = df18.append(df18H)
df18 = df18.append(df18I)
df18 = df18.append(df18J)
df18 = df18.append(df18K)

In [82]:
newCountry18 = []
for ele in country18:
    ele = ele.replace('Scotland','UK')
    newCountry18.append(ele)

In [29]:
country161718 = df161718['country']
country161718.size
# country.head()

1810

In [52]:
newCountry = []
for ele in country161718:
    ele = ele.replace('Scotland','UK')
    newCountry.append(ele)

In [83]:
tplCount = {}
isolates = {}
for idx, ele in enumerate(newCountry18):
    # print (idx, ele)
    if '-' in ele:
        tpl = ele.split('-')
        tpl = list(set(tpl))
        # exists international collboration
        if len(tpl) > 1:
        # Get all permutations of [1, 2, 3] 
            comb = combinations(tpl, 2) 
            for i in list(comb):
                if i not in tplCount:
                    tplCount[i] = 1
                else:
                    tplCount[i] += 1 
print (tplCount)
with open('./output/2018All.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in tplCount:
        val = tplCount[key]
        fout.write('\t'.join([key[0],key[1],str(val)]))
        fout.write('\n')

{('Korea', 'USA'): 11, ('Finland', 'UK'): 5, ('Germany', 'Denmark'): 6, ('Germany', 'Canada'): 4, ('Canada', 'USA'): 18, ('Korea', 'Canada'): 2, ('Israel', 'USA'): 1, ('Germany', 'Finland'): 4, ('Philippines', 'USA'): 1, ('France', 'Canada'): 8, ('Switzerland', 'Korea'): 1, ('Switzerland', 'USA'): 4, ('China', 'Hong Kong'): 3, ('Finland', 'Korea'): 2, ('Denmark', 'Switzerland'): 2, ('Germany', 'France'): 2, ('Germany', 'UK'): 8, ('France', 'UK'): 4, ('Germany', 'Austria'): 3, ('China', 'USA'): 15, ('Ireland', 'Finland'): 1, ('Ireland', 'UK'): 3, ('Sweden', 'UK'): 11, ('Italy', 'USA'): 1, ('Canada', 'UK'): 9, ('Israel', 'France'): 1, ('Hungary', 'USA'): 1, ('Germany', 'Australia'): 3, ('Germany', 'USA'): 10, ('Austria', 'USA'): 3, ('Netherlands', 'UK'): 3, ('Germany', 'Japan'): 3, ('USA', 'UK'): 8, ('Taiwan', 'China'): 1, ('Taiwan', 'Canada'): 1, ('Taiwan', 'USA'): 7, ('China', 'Canada'): 2, ('UK', 'USA'): 7, ('Switzerland', 'UK'): 3, ('Singapore', 'Canada'): 2, ('France', 'USA'): 4, ('

In [55]:
tplCount

{}

In [84]:
keyList161718 = list(tplCount)
ctyID = {}
count = 0
for pair in keyList161718:
    for ele in pair:
        if ele not in ctyID:
            ctyID[ele] = {'ID': count, 'color': ''}
            ctyID[ele]['ID'] = count
            if ele in Asia:
                ctyID[ele]['color'] = ctyColor['Asia']
            elif ele in NorthAmerica:
                ctyID[ele]['color'] = ctyColor['NorthAmerica']
            elif ele in LatinAmerica:
                ctyID[ele]['color'] = ctyColor['LatinAmerica']             
            elif ele in Europe:
                ctyID[ele]['color'] = ctyColor['Europe']
            elif ele in Oceanic:
                ctyID[ele]['color'] = ctyColor['Oceanic']
            elif ele in Africa:
                ctyID[ele]['color'] = ctyColor['Africa']
            count += 1

In [85]:
newtplCount = {}
for key in tplCount:
    newKey = (ctyID[key[0]]['ID'], ctyID[key[1]]['ID'])
    newtplCount[newKey] = tplCount[key]

In [87]:
with open('./output/2018AllEdgesNumbered.txt', 'w') as fout:
    fout.write('\t'.join(['C1','C2', 'weight']))
    fout.write('\n')
    for key in newtplCount:
        val = newtplCount[key]
        fout.write('\t'.join([str(key[0]),str(key[1]),str(val)]))
        fout.write('\n')

In [88]:
pubCount = {}
for ele in newCountry18:
        if '-' not in ele:
            if ele not in pubCount:
                pubCount[ele] = 1
            else:
                pubCount[ele] += 1
        else:
            lst = ele.split('-')
            for cty in lst:
                if cty not in pubCount:
                    pubCount[cty] = 1
                else:
                    pubCount[cty] += 1
for ele in pubCount:
    if ele in ctyID:
        ctyID[ele]['pubCount'] = pubCount[ele]

with open('./output/2018AllcountryID.txt', 'w') as fout:
    fout.write('Country' + '\t' + 'ID' + '\t' + 'Color' + '\t' + 'pubCount' + '\n')
    for key in ctyID:
        fout.write(key + '\t' + str(ctyID[key]['ID']) + '\t' + str(ctyID[key]['color']) + '\t' + str(ctyID[key]['pubCount']) + '\n')
print (ctyID)

{'Korea': {'ID': 0, 'color': '#fffac8', 'pubCount': 43}, 'USA': {'ID': 1, 'color': '#e6194B', 'pubCount': 523}, 'Finland': {'ID': 2, 'color': '#42d4f4', 'pubCount': 19}, 'UK': {'ID': 3, 'color': '#42d4f4', 'pubCount': 167}, 'Germany': {'ID': 4, 'color': '#42d4f4', 'pubCount': 108}, 'Denmark': {'ID': 5, 'color': '#42d4f4', 'pubCount': 30}, 'Canada': {'ID': 6, 'color': '#e6194B', 'pubCount': 89}, 'Israel': {'ID': 7, 'color': '#fffac8', 'pubCount': 6}, 'Philippines': {'ID': 8, 'color': '#fffac8', 'pubCount': 1}, 'France': {'ID': 9, 'color': '#42d4f4', 'pubCount': 37}, 'Switzerland': {'ID': 10, 'color': '#42d4f4', 'pubCount': 14}, 'China': {'ID': 11, 'color': '#fffac8', 'pubCount': 26}, 'Hong Kong': {'ID': 12, 'color': '#fffac8', 'pubCount': 8}, 'Austria': {'ID': 13, 'color': '#42d4f4', 'pubCount': 7}, 'Ireland': {'ID': 14, 'color': '#42d4f4', 'pubCount': 5}, 'Sweden': {'ID': 15, 'color': '#42d4f4', 'pubCount': 26}, 'Italy': {'ID': 16, 'color': '#42d4f4', 'pubCount': 2}, 'Hungary': {'ID': 

In [18]:
count = 0
for ele in country18:
    if '-' in ele:
        tpl = set(ele.split('-'))
        if len(tpl) > 1:
            count += 1
        
print (count)

208


In [28]:
df161718 = df1617.append(df18)

In [44]:
j = [1,2,3,4]
for i in j:
    if i ==1:
        i = 2
j

[1, 2, 3, 4]

In [150]:
c_usa = 0
c_usa_coll = 0

c_uk = 0
c_uk_coll = 0

c_ca = 0
c_ca_coll = 0

c_au = 0
c_au_coll = 0

c_dm = 0
c_dm_coll = 0

c_gm = 0
c_gm_coll = 0

c_fl = 0
c_fl_coll = 0

for ele in newCountry16:
    ele = set(ele.split('-'))
    #print (ele)
    if 'USA' in ele:
        c_usa += 1
        if len(ele) > 1:
            c_usa_coll += 1

    if 'UK' in ele:
        c_uk += 1
        if len(ele) > 1:
            c_uk_coll += 1
            
    if 'Canada' in ele:
        c_ca += 1
        if len(ele) > 1:
            c_ca_coll += 1
            
    if 'Australia' in ele:
        c_au += 1
        if len(ele) > 1:
            c_au_coll += 1
            
            
    if 'Germany' in ele:
        c_gm += 1
        if len(ele) > 1:
            c_gm_coll += 1
            
    if 'Denmark' in ele:
        c_dm += 1
        if len(ele) > 1:
            c_dm_coll += 1
            
    if 'Finland' in ele:
        c_fl += 1
        if len(ele) > 1:
            c_fl_coll += 1

print ('--------------USA------------')
print (c_usa, c_usa_coll) 
print (c_usa_coll/c_usa)
print ('--------------UK------------')
print (c_uk, c_uk_coll)
print (c_uk_coll/c_uk)
print ('--------------Canada------------')
print (c_ca, c_ca_coll)
print (c_ca_coll/c_ca)
print ('--------------Australia------------')
print (c_au, c_au_coll)
print (c_au_coll/c_au)
print ('--------------Germany------------')
print (c_gm, c_gm_coll)
print (c_gm_coll/c_gm)
print ('--------------Denmark------------')
print (c_dm, c_dm_coll)
print (c_dm_coll/c_dm)
print ('--------------Finland------------')
print (c_fl, c_fl_coll)
print (c_fl_coll/c_fl)

--------------USA------------
274 63
0.22992700729927007
--------------UK------------
99 40
0.40404040404040403
--------------Canada------------
57 33
0.5789473684210527
--------------Australia------------
11 2
0.18181818181818182
--------------Germany------------
43 21
0.4883720930232558
--------------Denmark------------
21 12
0.5714285714285714
--------------Finland------------
17 6
0.35294117647058826


In [151]:
c_usa = 0
c_usa_coll = 0

c_uk = 0
c_uk_coll = 0

c_ca = 0
c_ca_coll = 0

c_au = 0
c_au_coll = 0

c_dm = 0
c_dm_coll = 0

c_gm = 0
c_gm_coll = 0

c_fl = 0
c_fl_coll = 0

for ele in newCountry17:
    ele = set(ele.split('-'))
    #print (ele)
    if 'USA' in ele:
        c_usa += 1
        if len(ele) > 1:
            c_usa_coll += 1

    if 'UK' in ele:
        c_uk += 1
        if len(ele) > 1:
            c_uk_coll += 1
            
    if 'Canada' in ele:
        c_ca += 1
        if len(ele) > 1:
            c_ca_coll += 1
            
    if 'Australia' in ele:
        c_au += 1
        if len(ele) > 1:
            c_au_coll += 1
            
            
    if 'Germany' in ele:
        c_gm += 1
        if len(ele) > 1:
            c_gm_coll += 1
            
    if 'Denmark' in ele:
        c_dm += 1
        if len(ele) > 1:
            c_dm_coll += 1
            
    if 'Finland' in ele:
        c_fl += 1
        if len(ele) > 1:
            c_fl_coll += 1

print ('--------------USA------------')
print (c_usa, c_usa_coll) 
print (c_usa_coll/c_usa)
print ('--------------UK------------')
print (c_uk, c_uk_coll)
print (c_uk_coll/c_uk)
print ('--------------Canada------------')
print (c_ca, c_ca_coll)
print (c_ca_coll/c_ca)
print ('--------------Australia------------')
print (c_au, c_au_coll)
print (c_au_coll/c_au)
print ('--------------Germany------------')
print (c_gm, c_gm_coll)
print (c_gm_coll/c_gm)
print ('--------------Denmark------------')
print (c_dm, c_dm_coll)
print (c_dm_coll/c_dm)
print ('--------------Finland------------')
print (c_fl, c_fl_coll)
print (c_fl_coll/c_fl)

--------------USA------------
310 81
0.26129032258064516
--------------UK------------
98 40
0.40816326530612246
--------------Canada------------
57 33
0.5789473684210527
--------------Australia------------
17 5
0.29411764705882354
--------------Germany------------
55 22
0.4
--------------Denmark------------
19 13
0.6842105263157895
--------------Finland------------
17 12
0.7058823529411765


In [152]:
c_usa = 0
c_usa_coll = 0

c_uk = 0
c_uk_coll = 0

c_ca = 0
c_ca_coll = 0

c_au = 0
c_au_coll = 0

c_dm = 0
c_dm_coll = 0

c_gm = 0
c_gm_coll = 0

c_fl = 0
c_fl_coll = 0

for ele in newCountry18:
    ele = set(ele.split('-'))
    #print (ele)
    if 'USA' in ele:
        c_usa += 1
        if len(ele) > 1:
            c_usa_coll += 1

    if 'UK' in ele:
        c_uk += 1
        if len(ele) > 1:
            c_uk_coll += 1
            
    if 'Canada' in ele:
        c_ca += 1
        if len(ele) > 1:
            c_ca_coll += 1
            
    if 'Australia' in ele:
        c_au += 1
        if len(ele) > 1:
            c_au_coll += 1
            
    if 'Korea' in ele:
        c_kr += 1
        if len(ele) > 1:
            c_kr_coll += 1
            
    if 'Germany' in ele:
        c_gm += 1
        if len(ele) > 1:
            c_gm_coll += 1
            
    if 'Denmark' in ele:
        c_dm += 1
        if len(ele) > 1:
            c_dm_coll += 1
            
    if 'Finland' in ele:
        c_fl += 1
        if len(ele) > 1:
            c_fl_coll += 1

print ('--------------USA------------')
print (c_usa, c_usa_coll) 
print (c_usa_coll/c_usa)
print ('--------------UK------------')
print (c_uk, c_uk_coll)
print (c_uk_coll/c_uk)
print ('--------------Canada------------')
print (c_ca, c_ca_coll)
print (c_ca_coll/c_ca)
print ('--------------Australia------------')
print (c_au, c_au_coll)
print (c_au_coll/c_au)
print ('--------------Korea------------')
print (c_kr, c_kr_coll)
print (c_kr_coll/c_kr)
print ('--------------Germany------------')
print (c_gm, c_gm_coll)
print (c_gm_coll/c_gm)
print ('--------------Denmark------------')
print (c_dm, c_dm_coll)
print (c_dm_coll/c_dm)
print ('--------------Finland------------')
print (c_fl, c_fl_coll)
print (c_fl_coll/c_fl)

--------------USA------------
357 99
0.2773109243697479
--------------UK------------
109 56
0.5137614678899083
--------------Canada------------
68 46
0.6764705882352942
--------------Australia------------
19 11
0.5789473684210527
--------------Korea------------
307 206
0.6710097719869706
--------------Germany------------
73 43
0.589041095890411
--------------Denmark------------
30 19
0.6333333333333333
--------------Finland------------
18 15
0.8333333333333334


In [157]:
all_coll = 0
nonCoreColl = 0

cores = ['USA', 'UK', 'Canada', 'Germany', 'Denmark', 'Finland']
for ele in newCountry16:
    ele = set(ele.split('-'))
    #print (ele)
    #print ('--------------')
    if len(ele) > 1:
        all_coll += 1
        c = 0
        for core in cores:
            if core not in ele:
                c += 1
        if c == 6:
            nonCoreColl += 1
            #print (ele)
            #print ('****************')

print (all_coll, nonCoreColl)
print (nonCoreColl/all_coll)
                
        


135 10
0.07407407407407407


In [158]:
all_coll = 0
nonCoreColl = 0

cores = ['USA', 'UK', 'Canada', 'Germany', 'Denmark', 'Finland']
for ele in newCountry17:
    ele = set(ele.split('-'))
    #print (ele)
    #print ('--------------')
    if len(ele) > 1:
        all_coll += 1
        c = 0
        for core in cores:
            if core not in ele:
                c += 1
        if c == 6:
            nonCoreColl += 1
            #print (ele)
            #print ('****************')

print (all_coll, nonCoreColl)
print (nonCoreColl/all_coll)
                

156 11
0.07051282051282051


In [159]:
all_coll = 0
nonCoreColl = 0

cores = ['USA', 'UK', 'Canada', 'Germany', 'Denmark', 'Finland']
for ele in newCountry18:
    ele = set(ele.split('-'))
    #print (ele)
    #print ('--------------')
    if len(ele) > 1:
        all_coll += 1
        c = 0
        for core in cores:
            if core not in ele:
                c += 1
        if c == 6:
            nonCoreColl += 1
            #print (ele)
            #print ('****************')

print (all_coll, nonCoreColl)
print (nonCoreColl/all_coll)
                

208 13
0.0625
